# **Recherche d’hyperparamètres - Stratégie structurée et contrainte**

## **1. Objectif général**
Concevoir une **recherche d’hyperparamètres rigoureuse, efficace et reproductible** pour les agents du challenge *Permuted MNIST*, sous fortes contraintes matérielles et temporelles.  
L’approche repose sur une **méthodologie progressive et modulaire**, structurée autour de trois phases d’évaluation (A → B → C) avec **pruning adaptatif**.  
Elle vise à trouver le meilleur compromis entre **précision, stabilité et efficacité CPU**.

---

## **2. Contexte et contraintes**
- **CPU-only**, 2 threads maximum  
- **Mémoire ≤ 4 Go**  
- **Temps d’exécution ≤ 60 s / task** (init + train + test)

Ces contraintes excluent l’usage de modèles lourds (CNN, transformers) et motivent l’emploi d’**architectures MLP légères et optimisées CPU**.

---

## **3. Philosophie de la recherche**
1. Définir un **espace de recherche** pertinent centré sur les hyperparamètres influents.  
2. Explorer progressivement trois phases d’évaluation (A, B, C) de complexité croissante.  
3. Éliminer rapidement les modèles faibles grâce à un **pruning adaptatif**.  
4. Enregistrer systématiquement les résultats (CSV + JSON) pour assurer traçabilité et analyse.

Cette démarche reproduit une **approche scientifique contrôlée** : hypothèses → tests → validation.

---

## **4. Structure de la démarche**

| Phase | Objectif | # Tasks | Pruning | Description |
|:------|:----------|:--------|:---------|:-------------|
| **A — Exploration large** | Identifier les tendances globales | 3 | Oui | Sélection initiale rapide. |
| **B — Raffinement** | Confirmer les meilleures configs | 6 | Oui | Seuils plus stricts, meilleure stabilité. |
| **C — Validation finale** | Évaluer la robustesse | 10 | Non | Estimation finale (moyenne et écart-type). |

Chaque phase produit un rapport `.csv` et un résumé `.json`.

---

## **5. Architecture générale du pipeline**

Le pipeline repose sur quatre briques fondamentales :

| Brique | Rôle |
|:-------|:-----|
| **make_agent_factory** | Crée dynamiquement une fonction `make_agent(cfg)` à partir d’un module et d’une classe. |
| **eval_cfg** | Évalue une configuration sur plusieurs tasks avec règles de pruning (temps et accuracy). |
| **run_phase** | Exécute une phase (A/B/C), agrège les résultats et écrit un CSV. |
| **run_hparam_search** | Orchestration complète : A → B → C + tri des meilleurs modèles. |

Des fonctions utilitaires complètent le pipeline :
- `product_space` : génère toutes les combinaisons d’un espace de recherche.  
- `pick_top` : sélectionne les meilleures configurations.  
- `save_phase_csv` : sauvegarde les résultats intermédiaires.  

Enfin, des **builders** permettent de déclarer rapidement des espaces et règles cohérents :
- `hidden_from_lists`, `grid_MLP`
- `preset_tasks`, `preset_pruning`
- `make_experiment`, `run_and_report`


## **0) Setup & contraintes (à exécuter en premier)**



Cette première section initialise l’environnement du challenge et applique les contraintes d’exécution :

- **CPU-only**, 2 threads maximum  
- **Mémoire ≤ 4 Go**  
- **Temps ≤ 60 secondes** par task (initialisation + entraînement + test)

Le bloc ci-dessous :
1. importe les bibliothèques nécessaires,  
2. applique les limites matérielles du challenge,  
3. initialise la graine aléatoire,  
4. et charge l’environnement `PermutedMNISTEnv`.

In [ ]:
# --- Imports de base (sans hack de sys.path) ---

import os
import sys

try:
    base_path = os.path.dirname(__file__)
except NameError:
    base_path = os.getcwd()

# 1️⃣ Ajouter le dossier parent (un cran au-dessus)
parent_dir = os.path.abspath(os.path.join(base_path, '..'))
sys.path.append(parent_dir)

# 2️⃣ Ajouter le dossier parent du dossier parent (deux crans au-dessus)
two_up_dir = os.path.abspath(os.path.join(base_path, '..', '..'))
sys.path.append(two_up_dir)
import numpy as np
import time

# Import the environment and agents
from permuted_mnist.env.permuted_mnist import PermutedMNISTEnv

import json, csv, random, itertools, importlib, inspect
from pathlib import Path
from typing import Dict, List, Tuple, Callable, Iterable

print("✓ Imports successful")



SEED = 42
random.seed(SEED)
np.random.seed(SEED)

[limits] OMP/BLAS threads=2 | CUDA_VISIBLE_DEVICES=-1
[limits] RLIMIT_AS: soft=8589934591GB hard=8589934591GB
[limits] OMP/BLAS threads=2 | CUDA_VISIBLE_DEVICES=-1
[limits] RLIMIT_AS: soft=8589934591GB hard=8589934591GB



## **1) Fabrique d'agent générique**


**Objectif :** instancier dynamiquement un agent à partir de son module et de sa classe,  
sans modifier le reste du pipeline.

Cette fonction :
- charge dynamiquement un agent via `importlib`,
- filtre automatiquement les paramètres non supportés dans `__init__`,
- insère les valeurs par défaut (`seed`, `output_dim`),
- et renvoie une fonction `make_agent(cfg)` prête à l’emploi.

C’est la brique qui permet de tester différents agents sans changer la logique du pipeline.

In [15]:
def make_agent_factory(agent_spec: Tuple[str, str],
                       fixed_kwargs: Dict | None = None,
                       seed: int = 42) -> Callable[[Dict], object]:
    """
    Retourne une fonction make_agent(cfg) qui instancie l'agent défini par agent_spec
    en filtrant automatiquement les kwargs non supportés par __init__ de la classe.
    """
    if fixed_kwargs is None:
        fixed_kwargs = {}

    mod = importlib.import_module(agent_spec[0])
    AgentCls = getattr(mod, agent_spec[1])

    sig = inspect.signature(AgentCls.__init__)
    allowed = set(sig.parameters.keys()) - {"self"}

    def make_agent(cfg: Dict) -> object:
        kw = dict(fixed_kwargs); kw.update(cfg)
        # Valeurs par défaut utiles si absentes
        kw.setdefault("seed", seed)
        kw.setdefault("output_dim", 10)
        # Filtrage automatique
        kw = {k: v for k, v in kw.items() if k in allowed}
        return AgentCls(**kw)

    return make_agent


## **2) Génération d'espaces (grid) et utilitaires**


Ces fonctions gèrent la création, la sélection et la sauvegarde des configurations testées :

- **`product_space(space)`** : transforme un dictionnaire `{param: [valeurs...]}`  
  en liste de combinaisons d’hyperparamètres.
- **`pick_top(results, k)`** : trie les résultats selon un score pénalisé  
  (temps et pruning, puis accuracy).
- **`save_phase_csv(...)`** : sauvegarde les résultats d’une phase au format CSV  
  pour permettre l’audit et la traçabilité.

Ces briques rendent la recherche systématique, reproductible et analysable.

In [16]:

def product_space(space: Dict[str, Iterable]) -> List[Dict]:
    """
    space: {"param": [v1, v2, ...], "param2": [...]}
    -> liste de dicts (toutes les combinaisons, mélangées)
    """
    keys = list(space.keys())
    vals = [list(space[k]) for k in keys]
    out = []
    for combo in itertools.product(*vals):
        out.append({k: v for k, v in zip(keys, combo)})
    random.shuffle(out)
    return out

def pick_top(results: List[Dict], k: int) -> List[Dict]:
    """
    Classement pénalisé: d'abord pénalité prune, puis -accuracy, puis temps moyen.
    results: sorties de eval_cfg(...)
    """
    def key(r):
        pen = (1.0 if r["pruned_time"] else 0.0) + (0.5 if r["pruned_acc"] else 0.0)
        return (pen, -r["mean_acc"], r["mean_time"])
    return sorted(results, key=key)[:k]

def save_phase_csv(path: Path, results: List[Dict], param_keys: List[str]):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", newline="") as f:
        w = csv.writer(f)
        header = param_keys + ["mean_acc","std_acc","mean_time","total_time","n_tasks","pruned_time","pruned_acc"]
        w.writerow(header)
        for r in results:
            row = [r["config"].get(k, None) for k in param_keys]
            row += [r["mean_acc"], r["std_acc"], r["mean_time"], r["total_time"], r["n_tasks"],
                    r["pruned_time"], r["pruned_acc"]]
            w.writerow(row)


## **3) Évaluation d'une config avec pruning (multi-tâches)**


Cette fonction évalue une configuration donnée sur plusieurs tasks successives,  
en appliquant des **règles de pruning** (temps ou accuracy) pour interrompre les essais inutiles.

Étapes :
1. instanciation d’un agent via `make_agent(cfg)`,  
2. entraînement et évaluation sur chaque task,  
3. application des règles d’arrêt :
   - **temps** : dépassement du budget (`time_factor_stop × time_budget_s`),  
   - **accuracy** : performance insuffisante selon les seuils `A_first`, `A_mean`, etc.

Sortie : un dictionnaire contenant les moyennes et écarts-types de performance,  
les temps d’exécution, et les indicateurs de pruning :

| Clé | Signification |
|:----|:---------------|
| `mean_acc` | moyenne des accuracies sur les tasks évaluées |
| `std_acc` | écart-type des accuracies |
| `mean_time` | temps moyen par task |
| `pruned_time` | True si arrêt pour dépassement de temps |
| `pruned_acc` | True si arrêt pour performance insuffisante |

In [17]:
def eval_cfg(cfg: Dict,
             make_agent: Callable[[Dict], object],
             env_tasks: int,
             prune_rules: Dict,
             seed: int = 42,
             include_init_time: bool = False) -> Dict:
    """
    prune_rules attend:
      {
        "time_budget_s": 58.0,
        "time_factor_stop": 1.20,     # stop si t_task > factor * budget
        "phase": "A" / "B" / None,
        "A_first": 0.970, "A_mean": 0.970,
        "B_first": 0.980, "B_mean": 0.980
      }
    include_init_time:
      - False: on mesure train+predict (comme avant)
      - True : on mesure init+reset+train+predict (proche plateforme)
    """
    env = PermutedMNISTEnv(number_episodes=env_tasks)
    env.set_seed(seed)

    accs, times = [], []
    pruned_time = False
    pruned_acc = False
    phase = prune_rules.get("phase", None)

    t_id = 0
    while True:
        task = env.get_next_task()
        if task is None:
            break
        t_id += 1

        if include_init_time:
            t0 = time.time()
            agent = make_agent(cfg)     # on compte le coût d'instanciation
            agent.reset()
            t_start = t0
        else:
            agent = make_agent(cfg)
            agent.reset()
            t_start = time.time()

        agent.train(task["X_train"], task["y_train"])
        preds = agent.predict(task["X_test"])
        elapsed = time.time() - t_start

        acc = env.evaluate(preds, task["y_test"])
        accs.append(acc)
        times.append(elapsed)

        # Pruning temps
        if elapsed > prune_rules["time_factor_stop"] * prune_rules["time_budget_s"]:
            pruned_time = True
            break

        # Pruning accuracy
        if phase == "A":
            if len(accs) == 1 and accs[0] < prune_rules["A_first"]:
                pruned_acc = True; break
            if len(accs) >= 2 and np.mean(accs) < prune_rules["A_mean"]:
                pruned_acc = True; break
        if phase == "B":
            if len(accs) == 1 and accs[0] < prune_rules["B_first"]:
                pruned_acc = True; break
            if len(accs) >= 2 and np.mean(accs) < prune_rules["B_mean"]:
                pruned_acc = True; break

        if t_id >= env_tasks:
            break

    return {
        "config": cfg,
        "mean_acc": float(np.mean(accs) if accs else 0.0),
        "std_acc": float(np.std(accs) if accs else 0.0),
        "mean_time": float(np.mean(times) if times else 0.0),
        "total_time": float(np.sum(times)),
        "n_tasks": len(accs),
        "pruned_time": pruned_time,
        "pruned_acc": pruned_acc
    }


## **4) Orchestration d'une phase (A / B / C)**




Cette fonction exécute une phase complète de la recherche (A, B ou C).  
Elle enchaîne plusieurs évaluations `eval_cfg` sur la liste de configurations à tester.

**Fonctions principales :**
- `pretty_cfg(cfg)` : formate une configuration sous forme lisible pour les logs.  
- `run_phase(label, cfg_list, make_agent, env_tasks, prune_rules, out_csv, param_keys, include_init_time)` :
  - lance la boucle d’évaluation des configurations,
  - affiche un log clair (accuracy, temps, statut de pruning),
  - sauvegarde les résultats dans un CSV si `out_csv` est défini.

**Design “stateless” :** chaque phase est indépendante (A ne dépend pas directement de B),  
ce qui simplifie le debugging et la réutilisation du pipeline.

In [18]:
def pretty_cfg(cfg: dict, keys: list[str] | None = None) -> str:
    if keys is None:
        keys = sorted(cfg.keys())
    items = []
    for k in keys:
        v = cfg.get(k, None)
        if isinstance(v, (list, tuple)):
            v = tuple(v)
        items.append(f"{k}={v}")
    return "{ " + ", ".join(items) + " }"

def run_phase(label: str,
              cfg_list: list[dict],
              make_agent: Callable[[dict], object],
              env_tasks: int,
              prune_rules: dict,
              out_csv: Path | None = None,
              param_keys: list[str] | None = None,
              include_init_time: bool = False) -> list[dict]:
    print(f"\n=== Phase {label}: N={len(cfg_list)}, tasks={env_tasks}, phase={prune_rules.get('phase', None)} ===")
    results = []
    for i, cfg in enumerate(cfg_list, 1):
        r = eval_cfg(cfg,
                     make_agent=make_agent,
                     env_tasks=env_tasks,
                     prune_rules=prune_rules,
                     seed=SEED,
                     include_init_time=include_init_time)
        tag = " PRUNE[T]" if r["pruned_time"] else (" PRUNE[A]" if r["pruned_acc"] else "")
        hp_str = pretty_cfg(cfg, param_keys or list(cfg.keys()))
        print(f"[{label} {i}/{len(cfg_list)}] {hp_str}  ->  "
              f"acc={r['mean_acc']:.4f}±{r['std_acc']:.4f} | t={r['mean_time']:.1f}s | tasks={r['n_tasks']}{tag}")
        results.append(r)
    if out_csv and param_keys:
        save_phase_csv(out_csv, results, param_keys)
    return results

## **5) Pipeline complet prêt à l'emploi (A -> B -> C)**




Cette fonction orchestre toute la recherche d’hyperparamètres selon la stratégie progressive :

1. **Phase A — Exploration large**  
   Explore un grand nombre de configurations, élimine rapidement les modèles trop lents ou inefficaces.  
   → Résultats sauvegardés dans `phase_A.csv`.

2. **Phase B — Raffinement**  
   Relance uniquement les meilleures configs de A (top-k), avec des seuils de pruning plus stricts.  
   → Résultats dans `phase_B.csv`.

3. **Phase C — Validation finale**  
   Évalue les meilleures configs issues de B sur plus de tâches, sans pruning.  
   → Résultats dans `phase_C.csv`.

Enfin, la fonction écrit un **résumé JSON final** contenant les modèles les plus performants (TOP-3 par défaut).

**Paramètres clés :**
- `experiment` : dictionnaire complet décrivant l’expérience (agent, espace, règles, etc.)
- `include_init_time` : si True, inclut le coût d’instanciation de l’agent dans le temps total
- `verbose_top3` : si True, affiche les 3 meilleures configurations finales

In [19]:
def run_hparam_search(experiment: Dict, include_init_time: bool = False, verbose_top3: bool = True) -> Dict:
    """
    Lance A -> B -> C pour un espace de recherche donné.
    Paramètres clés dans `experiment` :
      - agent_spec: ("module.path", "AgentClass")
      - fixed_kwargs: kwargs fixes pour l'agent (ex: {"time_budget_s": 55.0, "output_dim": 10, "seed": 42})
      - search_space: dict param -> list de valeurs
      - n_A, n_B_keep, n_C_keep: tailles par phase
      - tasks: {"A": int, "B": int, "C": int}
      - prune: dict de règles de pruning pour A/B/C
      - outdir: Path où écrire les CSV/JSON
      - seed: int
    include_init_time: True pour compter init+reset+train+predict (proche plateforme).
    """
    outdir = experiment["outdir"]; outdir.mkdir(parents=True, exist_ok=True)
    param_keys = list(experiment["search_space"].keys())

    make_agent = make_agent_factory(
        agent_spec=experiment["agent_spec"],
        fixed_kwargs=experiment.get("fixed_kwargs", {}),
        seed=experiment.get("seed", 42)
    )

    all_cfgs = product_space(experiment["search_space"])

    # Phase A
    A_cfgs = all_cfgs[:experiment["n_A"]]
    A = run_phase("A", A_cfgs, make_agent,
                  env_tasks=experiment["tasks"]["A"],
                  prune_rules=experiment["prune"]["A"],
                  out_csv=outdir/"phase_A.csv", param_keys=param_keys,
                  include_init_time=include_init_time)

    # Phase B (top de A)
    B_cfgs = [r["config"] for r in pick_top(A, experiment["n_B_keep"])]
    B = run_phase("B", B_cfgs, make_agent,
                  env_tasks=experiment["tasks"]["B"],
                  prune_rules=experiment["prune"]["B"],
                  out_csv=outdir/"phase_B.csv", param_keys=param_keys,
                  include_init_time=include_init_time)

    # Phase C (top de B)
    C_cfgs = [r["config"] for r in pick_top(B, experiment["n_C_keep"])]
    C = run_phase("C", C_cfgs, make_agent,
                  env_tasks=experiment["tasks"]["C"],
                  prune_rules=experiment["prune"]["C"],
                  out_csv=outdir/"phase_C.csv", param_keys=param_keys,
                  include_init_time=include_init_time)

    final = sorted(C, key=lambda r: (-r["mean_acc"], r["mean_time"]))
    with open(outdir/"final_top.json", "w") as f:
        json.dump(final[:3], f, indent=2)

    if verbose_top3:
        def _cfg_slice(r):
            return {k: r['config'].get(k) for k in param_keys}
        print("\nTOP-3:", [
            (_cfg_slice(final[0]), final[0]["mean_acc"], final[0]["mean_time"]) if len(final)>0 else None,
            (_cfg_slice(final[1]), final[1]["mean_acc"], final[1]["mean_time"]) if len(final)>1 else None,
            (_cfg_slice(final[2]), final[2]["mean_acc"], final[2]["mean_time"]) if len(final)>2 else None,
        ])

    return {"A": A, "B": B, "C": C, "final": final}



Ces fonctions facilitent la construction d’espaces d’hyperparamètres cohérents et rapides à explorer.

- **`hidden_from_lists`** : génère automatiquement des architectures cachées décroissantes (H1 ≥ H2 ≥ H3).  
  Exemple : `[[1024, 512], [768, 384, 192]]`
- **`grid_MLP`** : assemble un espace de recherche complet pour MLP, incluant :
  - le nombre de couches cachées,
  - le dropout,
  - la taille de batch,
  - le learning rate,
  - le label smoothing,
  - le poids de régularisation.

Ces fonctions garantissent des combinaisons valides et homogènes pour la recherche.


In [ ]:
# ====  A) Builders d'espaces de recherche (génériques) ====

def hidden_from_lists(layer_value_lists: list[list[int]],
                      enforce_nonincreasing: bool = True) -> list[tuple[int, ...]]:
    """
    Exemple: [[1024, 1536], [512, 768]] -> [(1024,512), (1024,768), (1536,512), (1536,768)]
    Si enforce_nonincreasing=True, garde seulement h1>=h2>=... (utile pour 2L/3L).
    """
    import itertools
    combos = list(itertools.product(*layer_value_lists))
    if enforce_nonincreasing:
        combos = [c for c in combos if all(c[i] >= c[i+1] for i in range(len(c)-1))]
    return [tuple(c) for c in combos]

def grid_MLP(hidden_tuples: list[tuple[int, ...]],
             dropout: list[float],
             batch_size: list[int],
             learning_rate: list[float],
             label_smoothing: list[float] = (0.0, 0.05),
             max_epochs: list[int] = (10,),
             val_fraction: list[float] = (0.10,),
             weight_decay: list[float] = (1e-4,)) -> dict:
    """Construit le dict `search_space` attendu par run_hparam_search."""
    return {
        "hidden": hidden_tuples,
        "dropout": list(dropout),
        "batch_size": list(batch_size),
        "learning_rate": list(learning_rate),
        "label_smoothing": list(label_smoothing),
        "max_epochs": list(max_epochs),
        "val_fraction": list(val_fraction),
        "weight_decay": list(weight_decay),
    }





Ces fonctions définissent les paramètres par défaut pour la recherche multi-phases :

- **`preset_tasks(mode)`** : définit le nombre de configurations et de tâches par phase selon le mode choisi :  
  - `"sanity"` → test rapide pour debug,  
  - `"quick"` → configuration standard,  
  - `"full"` → exploration exhaustive.  

- **`preset_pruning(depth, strictness)`** : fixe les seuils d’arrêt anticipé selon :
  - la **profondeur du réseau** (1, 2 ou 3 couches),
  - et la **rigueur du pruning** (`loose`, `std`, `tight`).

Ces presets permettent d’adapter automatiquement la stratégie à la complexité de l’architecture.

In [ ]:
# ====  B) Presets souples pour tasks + pruning ====

def preset_tasks(mode: str = "quick") -> dict:
    """
    Taille d’échantillonnage par phase.
    """
    if mode == "sanity":
        return {"n_A": 6, "n_B_keep": 3, "n_C_keep": 2,
                "tasks": {"A": 3, "B": 6, "C": 10}}
    elif mode == "quick":
        return {"n_A": 30, "n_B_keep": 8, "n_C_keep": 3,
                "tasks": {"A": 3, "B": 6, "C": 10}}
    elif mode == "full":
        return {"n_A": 60, "n_B_keep": 10, "n_C_keep": 5,
                "tasks": {"A": 3, "B": 6, "C": 10}}
    else:
        raise ValueError("mode must be 'sanity' | 'quick' | 'full'")

def preset_pruning(depth: int, strictness: str = "std") -> dict:
    """
    depth ∈ {1,2,3}; strictness ∈ {'loose','std','tight'} (seuils d'accuracy et time).
    """
    tf = {"loose": 1.25, "std": 1.20 if depth==1 else (1.15 if depth==2 else 1.10), "tight": 1.10}
    A_first = {1: 0.960, 2: 0.962, 3: 0.963}[depth]
    A_mean  = {1: 0.965, 2: 0.968, 3: 0.969}[depth]
    B_first = {1: 0.975, 2: 0.977, 3: 0.978}[depth]
    B_mean  = {1: 0.978, 2: 0.980, 3: 0.981}[depth]
    return {
        "A": {"time_budget_s": 58.0, "time_factor_stop": tf[strictness], "phase": "A",
              "A_first": A_first, "A_mean": A_mean},
        "B": {"time_budget_s": 58.0, "time_factor_stop": tf[strictness], "phase": "B",
              "B_first": B_first, "B_mean": B_mean},
        "C": {"time_budget_s": 58.0, "time_factor_stop": tf[strictness], "phase": None}
    }


**Objectif :** assembler tous les composants d’une expérience complète dans un seul objet Python,
directement exploitable par `run_hparam_search`.

Cette fonction prend :
- les spécifications d’agent (`agent_spec`),
- l’espace de recherche (`search_space`),
- la configuration des phases (`tasks_cfg`),
- les règles de pruning (`prune_rules`),
- et d’éventuels paramètres fixes (`fixed_kwargs`).

Elle renvoie un dictionnaire normalisé contenant toutes les informations nécessaires à la recherche.

In [ ]:
# ====  C) Fabricant d'expérience générique ====

def make_experiment(agent_spec: tuple[str,str],
                    search_space: dict,
                    outdir: Path,
                    tasks_cfg: dict,
                    prune_rules: dict,
                    fixed_kwargs: dict | None = None,
                    seed: int = 42) -> dict:
    """
    Assemble l'objet `experiment` attendu par run_hparam_search.
    """
    if fixed_kwargs is None:
        fixed_kwargs = {}
    fixed_defaults = {"time_budget_s": 55.0, "output_dim": 10, "seed": seed}
    fixed_defaults.update(fixed_kwargs)

    return {
        "agent_spec": agent_spec,
        "fixed_kwargs": fixed_defaults,
        "search_space": search_space,
        "n_A": tasks_cfg["n_A"], "n_B_keep": tasks_cfg["n_B_keep"], "n_C_keep": tasks_cfg["n_C_keep"],
        "tasks": tasks_cfg["tasks"],
        "prune": prune_rules,
        "outdir": outdir,
        "seed": seed
    }





Cette fonction est une **interface simplifiée** pour lancer rapidement une expérience complète.

- Appelle `run_hparam_search` avec les bons paramètres,
- Sauvegarde automatiquement les résultats au format `.json`,
- Et affiche un **TOP-k** des meilleures configurations (par défaut TOP-3).

**Usage typique :**
```python
res = run_and_report("Demo", experiment=exp, topk=3)

In [ ]:
# ====  D) Runner compact (rapport + JSON) ====

def run_and_report(tag: str, experiment: dict, include_init_time: bool = True, topk: int = 3):
    res = run_hparam_search(experiment, include_init_time=include_init_time, verbose_top3=False)
    final = res["final"]
    if not final:
        print(f"\n[{tag}] Aucun modèle final (pruning partout ?)")
        return res
    print(f"\n[{tag}] TOP-{topk} (tri = acc desc, puis temps asc)")
    for i, r in enumerate(sorted(final, key=lambda x: (-x["mean_acc"], x["mean_time"]))[:topk], 1):
        print(f"#{i} cfg={r['config']} | acc={r['mean_acc']:.4f} | t={r['mean_time']:.1f}s")
    return res

# **Recherches d'hyperparamètres**

## **Mode d’emploi — Lancer une expérience de recherche d’hyperparamètres**

Cette section montre comment exécuter une recherche complète d’hyperparamètres en utilisant le pipeline modulaire défini précédemment.

L’approche repose sur une logique en cinq étapes simples.

---

### **Étape 1 - Déclarer l’agent à tester**
Spécifie :
- le module et la classe de l’agent à utiliser (`AGENT_SPEC`),
- les paramètres fixes communs à toutes les expériences (`FIXED_KW`).

---

### **Étape 2 - Définir l’espace de recherche**
Construit la grille d’hyperparamètres à explorer :
- architecture cachée (hidden layers),
- dropout, batch size, learning rate, label smoothing, etc.  
Utilise les fonctions **`hidden_from_lists`** et **`grid_MLP`** pour créer un espace cohérent.

---

### **Étape 3 - Configurer la taille des phases et les règles de pruning**
Adapte la complexité de l’expérience à ton objectif :
- Mode *sanity* : test rapide de validation,
- Mode *quick* : recherche standard,
- Mode *full* : exploration exhaustive.  

Ajuste également la profondeur du réseau et la rigueur du pruning selon :
- `depth` : nombre de couches cachées (1, 2, 3, ...),
- `strictness` : niveau de rigueur (`loose`, `std`, `tight`).

---

### **Étape 4 - Construire l’expérience complète**
Assemble tous les composants dans un objet **`experiment`** prêt à être utilisé avec :
- l’agent (`AGENT_SPEC`),
- l’espace de recherche (`search_space`),
- les règles de pruning (`prune_rules`),
- les tailles de phases (`tasks_cfg`),
- et le répertoire de sortie (`outdir`).

---

### **Étape 5 - Lancer la recherche et afficher les résultats**
Exécute automatiquement la séquence **A → B → C**, en sauvegardant les résultats CSV et le résumé JSON final.  
Le pipeline renvoie :
- la moyenne et l’écart-type des accuracies (`mean_acc`, `std_acc`),
- le temps moyen d’exécution par tâche (`mean_time`),
- les indicateurs de pruning (`pruned_time`, `pruned_acc`),
- et les fichiers produits : `phase_A.csv`, `phase_B.csv`, `phase_C.csv`, `final_top.json`.

---

### **Astuce**
Pour tester différentes profondeurs de MLP :
- modifie `depth` dans `preset_pruning()`,
- change la liste `hidden_tuples`,
- ajuste le dossier de sortie `outdir`.

Chaque expérience est totalement indépendante et reproductible.

## **MLP à 1 couche**

In [ ]:
AGENT_SPEC = ("models.MLP.agent_mlp_v3", "Agent")
OUTDIR = Path("./experiments/exp_1L")

hidden = hidden_from_lists([[512, 768, 1024, 1536, 2048]], enforce_nonincreasing=False)
space  = grid_MLP(
    hidden_tuples=hidden,
    dropout=[0.00, 0.05, 0.10],
    batch_size=[1024, 2048, 3072],
    learning_rate=[1e-3, 1.2e-3, 1.5e-3],
    label_smoothing=[0.0, 0.05]
)

tasks  = preset_tasks("quick")
prune  = preset_pruning(depth=1, strictness="std")

exp = make_experiment(AGENT_SPEC, space, OUTDIR, tasks, prune)
res_1L = run_and_report("1-LAYER quick", exp)


=== Phase A: N=30, tasks=3, phase=A ===
[A 1/30] { hidden=(1024,), dropout=0.05, batch_size=3072, learning_rate=0.001, label_smoothing=0.0, max_epochs=10, val_fraction=0.1, weight_decay=0.0001 }  ->  acc=0.9625±0.0006 | t=8.5s | tasks=2 PRUNE[A]
[A 2/30] { hidden=(512,), dropout=0.1, batch_size=3072, learning_rate=0.001, label_smoothing=0.0, max_epochs=10, val_fraction=0.1, weight_decay=0.0001 }  ->  acc=0.9544±0.0000 | t=4.8s | tasks=1 PRUNE[A]
[A 3/30] { hidden=(768,), dropout=0.0, batch_size=3072, learning_rate=0.001, label_smoothing=0.0, max_epochs=10, val_fraction=0.1, weight_decay=0.0001 }  ->  acc=0.9585±0.0000 | t=3.9s | tasks=1 PRUNE[A]
[A 4/30] { hidden=(2048,), dropout=0.0, batch_size=2048, learning_rate=0.0012, label_smoothing=0.05, max_epochs=10, val_fraction=0.1, weight_decay=0.0001 }  ->  acc=0.9814±0.0007 | t=10.4s | tasks=3
[A 5/30] { hidden=(1536,), dropout=0.05, batch_size=1024, learning_rate=0.001, label_smoothing=0.05, max_epochs=10, val_fraction=0.1, weight_decay

## **MLP à 2 couches**

In [ ]:
AGENT_SPEC = ("models.MLP.agent_mlp_v3", "Agent")
OUTDIR = Path("./experiments/exp_2L")

hidden = hidden_from_lists(
    [[768, 1024, 1280, 1536],   # H1
     [256, 384, 512, 768]],     # H2
    enforce_nonincreasing=True
)
space = grid_MLP(
    hidden_tuples=hidden,
    dropout=[0.00, 0.05, 0.10],
    batch_size=[1024, 2048, 3072],
    learning_rate=[9e-4, 1.0e-3, 1.2e-3],
    label_smoothing=[0.0, 0.05]
)

tasks = preset_tasks("quick")
prune = preset_pruning(depth=2, strictness="std")

exp = make_experiment(AGENT_SPEC, space, OUTDIR, tasks, prune)
res_2L = run_and_report("2-LAYER quick", exp)


=== Phase A: N=30, tasks=3, phase=A ===
[A 1/30] { hidden=(1536, 384), dropout=0.05, batch_size=3072, learning_rate=0.0009, label_smoothing=0.0, max_epochs=10, val_fraction=0.1, weight_decay=0.0001 }  ->  acc=0.9722±0.0013 | t=15.1s | tasks=3
[A 2/30] { hidden=(768, 256), dropout=0.1, batch_size=1024, learning_rate=0.0009, label_smoothing=0.0, max_epochs=10, val_fraction=0.1, weight_decay=0.0001 }  ->  acc=0.9796±0.0009 | t=8.3s | tasks=3
[A 3/30] { hidden=(1024, 512), dropout=0.0, batch_size=2048, learning_rate=0.001, label_smoothing=0.05, max_epochs=10, val_fraction=0.1, weight_decay=0.0001 }  ->  acc=0.9815±0.0002 | t=9.5s | tasks=3
[A 4/30] { hidden=(1536, 384), dropout=0.0, batch_size=1024, learning_rate=0.001, label_smoothing=0.05, max_epochs=10, val_fraction=0.1, weight_decay=0.0001 }  ->  acc=0.9850±0.0004 | t=11.2s | tasks=3
[A 5/30] { hidden=(1280, 384), dropout=0.1, batch_size=3072, learning_rate=0.0009, label_smoothing=0.0, max_epochs=10, val_fraction=0.1, weight_decay=0.0

## **MLP à 3 couches**

In [ ]:
AGENT_SPEC = ("models.MLP.agent_mlp_v3", "Agent")
OUTDIR = Path("./experiments/exp_3L")

hidden = hidden_from_lists(
    [[896, 1024, 1280],  # H1
     [384, 512, 640],    # H2
     [192, 256, 320]],   # H3
    enforce_nonincreasing=True
)
space = grid_MLP(
    hidden_tuples=hidden,
    dropout=[0.05, 0.10],
    batch_size=[1024, 2048, 3072],
    learning_rate=[8e-4, 9e-4, 1.0e-3],
    label_smoothing=[0.0, 0.05]
)

tasks = preset_tasks("quick")
prune = preset_pruning(depth=3, strictness="std")

exp = make_experiment(AGENT_SPEC, space, OUTDIR, tasks, prune)
res_3L = run_and_report("3-LAYER quick", exp)


=== Phase A: N=30, tasks=3, phase=A ===
[A 1/30] { hidden=(1024, 384, 320), dropout=0.05, batch_size=1024, learning_rate=0.001, label_smoothing=0.05, max_epochs=10, val_fraction=0.1, weight_decay=0.0001 }  ->  acc=0.9837±0.0013 | t=15.2s | tasks=3
[A 2/30] { hidden=(1024, 640, 320), dropout=0.1, batch_size=2048, learning_rate=0.0009, label_smoothing=0.05, max_epochs=10, val_fraction=0.1, weight_decay=0.0001 }  ->  acc=0.9817±0.0001 | t=17.4s | tasks=3
[A 3/30] { hidden=(1280, 640, 320), dropout=0.1, batch_size=1024, learning_rate=0.0009, label_smoothing=0.0, max_epochs=10, val_fraction=0.1, weight_decay=0.0001 }  ->  acc=0.9804±0.0013 | t=20.0s | tasks=3
[A 4/30] { hidden=(1024, 640, 256), dropout=0.1, batch_size=1024, learning_rate=0.0009, label_smoothing=0.0, max_epochs=10, val_fraction=0.1, weight_decay=0.0001 }  ->  acc=0.9803±0.0005 | t=16.0s | tasks=3
[A 5/30] { hidden=(1280, 640, 256), dropout=0.05, batch_size=1024, learning_rate=0.0009, label_smoothing=0.05, max_epochs=10, val